<a href="https://colab.research.google.com/github/asheta66/Generative-AI-2/blob/main/Classification_using_Autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Set the directory for the data stored on Google Drive
data_dir = '/content/drive/My Drive/data2'

# Define image size and batch size for data generators
img_size = (128, 128)  # Adjust according to your data
batch_size = 32

# Set up data generators for training and validation with 20% validation split
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = datagen.flow_from_directory(data_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical', subset='training')
val_generator = datagen.flow_from_directory(data_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical', subset='validation')

# Determine the number of classes from the data generator
num_classes = len(train_generator.class_indices)

# Define the Autoencoder model structure
input_img = Input(shape=(img_size[0], img_size[1], 3))

# Encoder part
x = Flatten()(input_img)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
encoded = Dense(64, activation='relu')(x)

# Decoder part
x = Dense(128, activation='relu')(encoded)
x = Dense(256, activation='relu')(x)
x = Dense(img_size[0] * img_size[1] * 3, activation='sigmoid')(x)
decoded = Reshape((img_size[0], img_size[1], 3))(x)

# Compile the Autoencoder model
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

# Print the summary of the Autoencoder model
autoencoder.summary()

# Train the Autoencoder model
history = autoencoder.fit(train_generator, epochs=50, validation_data=val_generator)

# Extract the encoder part of the Autoencoder for feature extraction
encoder = Model(inputs=input_img, outputs=encoded)

# Define the classification model using the encoded features
encoded_input = Input(shape=(64,))
x = Dense(64, activation='relu')(encoded_input)
x = Dense(num_classes, activation='softmax')(x)
classifier = Model(inputs=encoded_input, outputs=x)

# Compile the classifier model with categorical crossentropy loss
classifier.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
classifier.summary()

# Generate encoded features for training and validation data
train_features = encoder.predict(train_generator)
val_features = encoder.predict(val_generator)

# Get the corresponding labels and convert them to one-hot encoding
train_labels = train_generator.classes
val_labels = val_generator.classes

train_labels = to_categorical(train_labels, num_classes=num_classes)
val_labels = to_categorical(val_labels, num_classes=num_classes)

# Train the classifier model using the encoded features
history_classifier = classifier.fit(train_features, train_labels, epochs=50, validation_data=(val_features, val_labels))

# Evaluate the classifier using the validation data
val_predictions = np.argmax(classifier.predict(val_features), axis=1)

# Create and display the confusion matrix
conf_matrix = confusion_matrix(val_generator.classes, val_predictions)
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Print the classification report for precision, recall, and F1-score
print(classification_report(val_generator.classes, val_predictions, target_names=val_generator.class_indices.keys()))

# Plot the training and validation loss curves for the Autoencoder
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Autoencoder Loss')
plt.legend()
plt.show()

# Plot the training and validation accuracy curves for the classifier
plt.plot(history_classifier.history['accuracy'], label='train_acc')
plt.plot(history_classifier.history['val_accuracy'], label='val_acc')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Classifier Accuracy')
plt.legend()
plt.show()
